In [ ]:
# Generate

In [1]:
from sentence_transformers import CrossEncoder
import logging
import os
import pandas as pd
import torch, gc
from datetime import datetime
import argparse
import matplotlib.pyplot as plt
# from financerag.rerank import CrossEncoderReranker
# from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder, BM25Retriever, HybridRetriever
from financerag.tasks import ConvFinQA, FinanceBench, FinDER, FinQA, FinQABench, MultiHiertt, TATQA
from financerag.generate import OpenAIGenerator, CustomGenerator
logging.basicConfig(level=logging.INFO)

torch.cuda.set_device(0)

/opt/conda/envs/DSC/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
final_rank = pd.read_csv('./data/final_submission.csv', index_col=False)
final_rank

,query_id,corpus_id
0,qd4982518,dd4c4fb38
1,qd4982518,dd4c4ef80
2,qd4982518,dd4c4f7aa
3,qd4982518,dd4ba3af4
4,qd4982518,dd4bf84f0
...,...,...
46705,q1a73d3ea,d1b3b712a
46706,q1a73d3ea,d1b35f90c
46707,q1a73d3ea,d1a73d426
46708,q1a73d3ea,d1b3858e6


In [3]:
convfinqa_task = ConvFinQA()
finbench_task = FinanceBench()
finder_task = FinDER()
finqa_task = FinQA()
finqabench_task = FinQABench()
multih_task = MultiHiertt()
tatqa_task = TATQA()

changed


In [8]:
finqabench_task.queries.values()

TypeError: 'dict_values' object is not subscriptable

In [13]:
import numpy as np
random_data = {
    "Column1": np.random.randint(1, 101, 100),  # 1~100 사이의 랜덤 정수
    "Column2": np.random.random(100)           # 0~1 사이의 랜덤 실수
}

# 데이터프레임 생성
df = pd.DataFrame(random_data)
df['qu'] = finqabench_task.queries.values()
df

,Column1,Column2,qu
0,3,0.889597,What is the redemption price for the 0.875% 20...
1,39,0.095079,What is the maturity date of the 0.875% 2025 N...
2,83,0.955969,What are the conditions under which the Compan...
3,12,0.557517,What is the Company's investment policy and st...
4,81,0.967048,What is the purpose of the Company's internal ...
...,...,...,...
95,82,0.486604,What risks are associated with the Company's i...
96,77,0.206261,What percentage of outstanding shares must vot...
97,93,0.724806,What is the minimum percentage of shares requi...
98,3,0.323131,What was the approximate dollar value of share...


In [4]:
convfinqa_results = final_rank.iloc[:4210]
finbench_results = final_rank.iloc[4210:6370]
finder_results = final_rank.iloc[6370:17870]
finqa_results = final_rank.iloc[17870:18870]
finqabench_results = final_rank.iloc[18870:20370]
multih_results = final_rank.iloc[20370:30110]
tatqa_results = final_rank.iloc[30110:]

In [5]:
confinqa_dict = {}
for key in convfinqa_results['query_id'].unique():
    group = convfinqa_results[convfinqa_results['query_id'] == key]
    confinqa_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

finbench_dict = {}
for key in finbench_results['query_id'].unique():
    group = finbench_results[finbench_results['query_id'] == key]
    finbench_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

finder_dict = {}
for key in finder_results['query_id'].unique():
    group = finder_results[finder_results['query_id'] == key]
    finder_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

finqa_dict = {}
for key in finqa_results['query_id'].unique():
    group = finqa_results[finqa_results['query_id'] == key]
    finqa_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

finqabench_dict = {}
for key in finqabench_results['query_id'].unique():
    group = finqabench_results[finqabench_results['query_id'] == key]
    finqabench_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

multih_dict = {}
for key in multih_results['query_id'].unique():
    group = multih_results[multih_results['query_id'] == key]
    multih_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

tatqa_dict = {}
for key in tatqa_results['query_id'].unique():
    group = tatqa_results[tatqa_results['query_id'] == key]
    tatqa_dict[key] = {value: idx for idx, value in enumerate(group['corpus_id'], start=1)}

In [6]:
convfinqa_task.rerank_results = confinqa_dict
finbench_task.rerank_results = finbench_dict
finder_task.rerank_results = finder_dict
finqa_task.rerank_results = finqa_dict
finqabench_task.rerank_results = finqabench_dict
multih_task.rerank_results = multih_dict
tatqa_task.rerank_results = tatqa_dict

In [7]:
example_messages = {
    "q1": {
        "query": "인공지능이 뭐야?",
        "document": "인공지능(AI)은 컴퓨터가 인간의 지적 능력을 모방하는 기술이다.",
        "prompt": "아래 문서를 참조해서 사용자의 질문에 답변해줘."
    },
    "q2": {
        "query": "Mistral 모델에 대해 알려줘",
        "document": "Mistral은 메타 AI에서 선보인 모델로 경량화와 효율성이 특징이다.",
        "prompt": "주어진 문서를 요약하여 사용자 질문에 답변을 작성해줘."
    }
}

generator = CustomGenerator("meta-llama/Llama-3.2-1B-Instruct")
answers = generator.generation(example_messages, max_new_tokens=128, temperature=0.7)

print("===== 결과 =====")
for q_id, ans in answers.items():
    print(f"- Query ID: {q_id}\n  Generated Answer: {ans}\n")

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
INFO:financerag.generate.generator:총 2개의 쿼리에 대해 32개 프로세스를 활용해 답변 생성을 시작합니다.


HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Llama-3.2-1B-Instruct/v1/chat/completions (Request ID: bZ3G_-i_jmmadt_HBu7T9)

Failed to deserialize the JSON body into the target type: messages: invalid type: map, expected a sequence at line 1 column 58